# Analyze Facebook Data Using IBM Watson and IBM Data Platform

This is a three-part notebook written in `Python_3.5` meant to show how anyone can enrich and analyze a combined dataset of unstructured and strucutured information with IBM Watson and IBM Data Platform. For this example we are using a standard Facebook Analytics export which features texts from posts, articles and thumbnails, along with standard performance metrics such as likes, shares, and impressions. 

1.  First, we use the Watson Tone Analyzer Service to enrich the Facebook Posts by pulling out `Emotion Tones` and related `Keywords`. 

2.  We will prep the data for analysis and visualization The end result will be a Pandas DataFrames that will contain the results of the analysis

3.  Finally, we will include services from IBM's Data Platform, including IBM's own data visualization library PixieDust, to analyze the data and visualize our results.

<a id="part1"></a>
#  Part I - Enrich
<a id='setup'></a> 
## 1. Setup
<a id="setup1"></a>
### 1.1 Install Latest Watson Developer Cloud and Beautiful Soup Packages

In [ ]:
!pip install --upgrade watson-developer-cloud

In [ ]:
!pip install --upgrade beautifulsoup4

If WDC or BS4 was just installed or upgraded, <span style="color: red">restart the kernel</span> before continuing

<a id="pixie"></a>
### 1.2 Install PixieDust and Wordcloud Libraries
This notebook provides an overview of how to use the PixieDust Library to analyze and visualize various data sets. If you are new to PixieDust or would like to learn more about the library, please go to this [Introductory Notebook](https://apsportal.ibm.com/exchange/public/entry/view/5b000ed5abda694232eb5be84c3dd7c1) or visit the [PixieDust Github](https://ibm-cds-labs.github.io/pixiedust/). The `Setup` section for this notebook uses instructions from the [Intro To PixieDust](https://github.com/ibm-cds-labs/pixiedust/blob/master/notebook/Intro%20to%20PixieDust.ipynb) notebook

To ensure you are running the latest version of PixieDust uncomment and run the following cell. Do not run this cell if you installed PixieDust locally from source and want to continue to run PixieDust from source.

In [ ]:
!pip install --user --upgrade pixiedust
!pip install wordcloud

<a id="setup2"></a>
### 1.3 Import Packages and Libraries
To check if you have package already installed, open new cell and write: *help.('Package Name')*

In [ ]:
import json
import sys
import watson_developer_cloud
from watson_developer_cloud import ToneAnalyzerV3, VisualRecognitionV3
import watson_developer_cloud.natural_language_understanding.features.v1 as features

import operator
from functools import reduce
from io import StringIO
import numpy as np
from bs4 import BeautifulSoup as bs
from operator import itemgetter
from os.path import join, dirname
import pandas as pd
import numpy as np
import requests
import pixiedust

<a id='setup3'></a>
### 1.4 Add Service Credentials From Bluemix for Watson Services

To create your own service and API keys for either NLU or Tone Analyzer go to the Watson Services on [Bluemix](https://www.ibm.com/cloud-computing/bluemix/).

After creating a service for NLU and Tone Analyzer, replace the credentials in the section below

###  <span style="color: red"> _User Input_</span> 

In [ ]:
nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2017-02-27',
                                                            username='bb8fa0e8-0e9d-4aea-acce-db7729cd7bdd',
                                                            password='n8yqbfFjjALL')
tone_analyzer = ToneAnalyzerV3(version='2016-05-19',
                               username='d69ea307-c1e9-44f0-aa56-9186fa25ca8e',
                               password='0EYsQzauIOzh')

visual_recognition = VisualRecognitionV3('2016-05-20', api_key='444d34887cf8d312596a852d862db5ccb68cc6c6')

<a id='load'></a> 
## 2. Load Data

### 2.1 Load data from Object Storage
IBM® Object Storage for Bluemix® provides provides you with access to a fully provisioned Swift Object Storage account to manage your data. Object Storage uses OpenStack Identity (Keystone) for authentication and can be accessed directly by using [OpenStack Object Storage (Swift) API v3](http://developer.openstack.org/api-ref-identity-v3.html#credentials-v3). 



###  <span style="color: red"> _User Input_</span> 

Insert data you want to enrich by clicking on the 1001 icon on the upper right hand of the screen. Click "Insert to code" under the file you want to enrich. The make sure you've clicked the cell below and then choose "Insert Pandas DataFrame."

In [ ]:
# The code was removed by DSX for sharing.

In [ ]:
# insert pandas dataframe

###  <span style="color: red"> _User Input_</span> 

In [ ]:
#Make sure this equals the variable above.
df = df_data_4
df.head()

###  <span style="color: red"> _User Input_</span> 

Put in the credentials for the file you want to enrich by clicking on the 1001 icon on the upper right hand of the screen. Click the cell below, then click "Insert to code" under the file you want to enrich. Choose "Insert Credentials." **CHANGE THE NAME TO `credentials_1`**

In [ ]:
# The code was removed by DSX for sharing.

###  <span style="color: red"> _User Input_</span> 

In [ ]:
#choose any name to save your file
localfilename = 'OutputFile.csv'

<a id='prepare'></a>
## 3. Prepare Data



<a id='prepare1'></a>
###  3.1 Data Cleansing with Python
Renaming columns, removing noticable noise in the data, pulling out URLs and appending to a new column to run through NLU

In [ ]:
df.rename(columns={'Post Message': 'Text'}, inplace=True)

In [ ]:
df = df.drop([0])
df.head()

In [ ]:
#Seperate links from posts

df_http= df["Text"].str.partition("http")
df_www = df["Text"].str.partition("www")

#combine delimiters with actual links
df_http["Link"] = df_http[1].map(str) + df_http[2]
df_www["Link1"] = df_www[1].map(str) + df_www[2]

#include only Link columns 
df_http.drop(df_http.columns[0:3], axis=1, inplace = True)
df_www.drop(df_www.columns[0:3], axis=1, inplace = True)

#merge http and www dataframes
dfmerge = pd.concat([df_http, df_www], axis=1)

#the following steps will allow you to merge data columns from the left to the right
dfmerge = dfmerge.apply(lambda x: x.str.strip()).replace('', np.nan)

#use fillna to fill any blanks with the Link1 column
dfmerge["Link"].fillna(dfmerge["Link1"], inplace = True)

#delete Link1 (www column)
dfmerge.drop("Link1", axis=1, inplace = True)

#combine Link data frame 
df = pd.concat([dfmerge,df], axis = 1)

# make sure text column is a string
# df["Text"] = str(df["Text"])

#strip links from Text column
#df['Text'] = df['Text'].apply(lambda x: x.split('http')[0])
#df['Text'] = df['Text'].apply(lambda x: x.split('www')[0])

df.head()

<a id='enrich'></a> 
## 4. Enrichment Time!
<a id='nlupost'></a>
###  4.1 NLU for the Post Text
Below uses Natural Language Understanding to iterate through each post and extract the enrichment features we want to use in our future analysis.

Each feature we extract will be appended to the `.csv` in a new column we determine at the end of this script. If you want to run this same script for the other columns, define `free_form_responses` to the column name, if you are using URLs, change `text=response` parameter to `url=response`, and update the new column names as you see fit. 

In [ ]:
# Extract the free form text response from the data frame
# If you are using this script for a diff CSV, you will have to change this column name
free_form_responses = df['Text']
# define the list of enrichments to apply
# if you are modifying this script add or remove the enrichments as needed
f = [features.Entities(), features.Keywords(),features.Emotion(),features.Sentiment()]#'typed-rels'

# Create a list to store the enriched data
overallSentimentScore = []
overallSentimentType = []
highestEmotion = []
highestEmotionScore = []
kywords = []
entities = []

# Go thru every reponse and enrich the text using NLU
for idx, response in enumerate(free_form_responses):
    #print("Processing record number: ", idx, " and text: ", response)
    try:
        enriched_json = json.loads(json.dumps(nlu.analyze(text=response, features=f)))
        #print(enriched_json)

        # get the SENTIMENT score and type
        if 'sentiment' in enriched_json:
            if('score' in enriched_json['sentiment']["document"]):
                overallSentimentScore.append(enriched_json["sentiment"]["document"]["score"])
            else:
                overallSentimentScore.append('0')

            if('label' in enriched_json['sentiment']["document"]):
                overallSentimentType.append(enriched_json["sentiment"]["document"]["label"])
            else:
                overallSentimentType.append('0')

        # read the EMOTIONS into a dict and get the key (emotion) with maximum value
        if 'emotion' in enriched_json:
            me = max(enriched_json["emotion"]["document"]["emotion"].items(), key=operator.itemgetter(1))[0]
            highestEmotion.append(me)
            highestEmotionScore.append(enriched_json["emotion"]["document"]["emotion"][me])

        else:
            highestEmotion.append("")
            highestEmotionScore.append("")

        #iterate and get KEYWORDS with a confidence of over 50%
        if 'keywords' in enriched_json:
            #print((enriched_json['keywords']))
            tmpkw = []
            for kw in enriched_json['keywords']:
                if(float(kw["relevance"]) >= 0.5):
                    #print("kw is: ", kw, "and val is ", kw["text"])
                    tmpkw.append(kw["text"])#str(kw["text"]).strip('[]')
            #convert multiple keywords in a list to a string
            if(len(tmpkw) > 1):
                tmpkw = "".join(reduce(lambda a, b: a + ', ' + b, tmpkw))
            elif(len(tmpkw) == 0):
                tmpkw = ""
            else:
                tmpkw = "".join(reduce(lambda a, b='': a + b , tmpkw))
            kywords.append(tmpkw)
        else:
            kywords.append("")
            
        #iterate and get Entities with a confidence of over 30%
        if 'entities' in enriched_json:
            #print((enriched_json['entities']))
            tmpent = []
            for ent in enriched_json['entities']:
                
                if(float(ent["relevance"]) >= 0.3):
                    tmpent.append(ent["type"])
            #convert multiple concepts in a list to a string
            if(len(tmpent) > 1):
                tmpent = "".join(reduce(lambda a, b: a + ', ' + b, tmpent))
            elif(len(tmpent) == 0):
                tmpent = ""
            else:
                tmpent = "".join(reduce(lambda a, b='': a + b , tmpent))
            entities.append(tmpent)
        else:
            entities.append("")    
            
    except:
        # catch *all* exceptions
        e = sys.exc_info()[0]
        overallSentimentScore.append(' ')
        overallSentimentType.append(' ')
        highestEmotion.append(' ')
        highestEmotionScore.append(' ')
        kywords.append(' ')
        entities.append(' ')
        pass
    
# Create columns from the list and append to the dataframe
if highestEmotion:
    df['TextHighestEmotion'] = highestEmotion
if highestEmotionScore:
    df['TextHighestEmotionScore'] = highestEmotionScore

if overallSentimentType:
    df['TextOverallSentimentType'] = overallSentimentType
if overallSentimentScore:
    df['TextOverallSentimentScore'] = overallSentimentScore

df['TextKeywords'] = kywords
df['TextEntities'] = entities
df.head(50)
#df.info()

After we extract all of the Keywords and Entities from each Post, we have a column with multiple Keywords, and Entities separated by commas. For our Analysis in Part II we wanted also wanted the top Keyword and Entity for each Post. Because of this, we added two new columns to capture the `MaxTextKeyword` and `MaxTextEntity`

In [ ]:
#choose first of Keywords,Concepts, Entities
df["MaxTextKeywords"] = df["TextKeywords"].apply(lambda x: x.split(',')[0])
df["MaxTextEntity"] = df["TextEntities"].apply(lambda x: x.split(',')[0])
#df.head()

 <a id='tonepost'></a> 
### 4.4 Tone Analyzer for Post Text

In [ ]:
# Extract the free form text response from the data frame
# If you are using this script for a diff CSV, you will have to change this column name
free_form_responses = df['Text']

#Create a list to store the enriched data

highestEmotionTone = []
emotionToneScore = []

languageToneScore = []
highestLanguageTone = []

socialToneScore = []
highestSocialTone = []


for idx, response in enumerate(free_form_responses):
    #print("Processing record number: ", idx, " and text: ", response)
    try:
        enriched_json = json.loads(json.dumps(tone_analyzer.tone(text=response)))
        #print(enriched_json)
        
        if 'tone_categories' in enriched_json['document_tone']:
            me = max(enriched_json["document_tone"]["tone_categories"][0]["tones"], key = itemgetter('score'))['tone_name']      
            highestEmotionTone.append(me)
            you = max(enriched_json["document_tone"]["tone_categories"][0]["tones"], key = itemgetter('score'))['score']
            emotionToneScore.append(you)
            
            me1 = max(enriched_json["document_tone"]["tone_categories"][1]["tones"], key = itemgetter('score'))['tone_name']      
            highestLanguageTone.append(me1)
            you1 = max(enriched_json["document_tone"]["tone_categories"][1]["tones"], key = itemgetter('score'))['score']
            languageToneScore.append(you1)
            
            me2 = max(enriched_json["document_tone"]["tone_categories"][2]["tones"], key = itemgetter('score'))['tone_name']      
            highestSocialTone.append(me2)
            you2 = max(enriched_json["document_tone"]["tone_categories"][2]["tones"], key = itemgetter('score'))['score']
            socialToneScore.append(you2)
            
            
            
    except:
        # catch *all* exceptions
        e = sys.exc_info()[0]
        emotionToneScore.append(' ')
        highestEmotionTone.append(' ')
        languageToneScore.append(' ')
        highestLanguageTone.append(' ')
        socialToneScore.append(' ')
        highestSocialTone.append(' ')
        pass
    
if highestEmotionTone:
    df['highestEmotionTone'] = highestEmotionTone    
if emotionToneScore:
    df['emotionToneScore'] = emotionToneScore
    
if languageToneScore:
    df['languageToneScore'] = languageToneScore
if highestLanguageTone:
    df['highestLanguageTone'] = highestLanguageTone
    
if highestSocialTone:
    df['highestSocialTone'] = highestSocialTone    
if socialToneScore:
    df['socialToneScore'] = socialToneScore 
    
df.head(50)
#df.info()

 <a id='write'></a>
## Enrichment is now COMPLETE!
<a id='write1'></a> 
Last step is to write and save the enriched dataframe to SoftLayer's Object Storage.

Since we already created the `localfilename` variable in the Setup stage and defined the necessary credentials, this snippet will work for all new files and does not need to be changed.

In [ ]:
def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r',encoding="utf-8")
    my_data = f.read()
    data_to_send = my_data.encode("utf-8")
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    #print(resp1_body)
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/',  local_file_name])
                            print(url2)
    s_subject_token = resp1.headers['x-subject-token']
    #print(s_subject_token)
    #print(credentials['container'])
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = data_to_send )
    print(resp2)

In [ ]:
#choose any name to save your file
df.to_csv('OutputFile.csv',index=False)

<a id='write2'></a> Make sure to change the "credential" argument below matches the variable name of the credentials you imported in the Setup Phase.

In [ ]:
put_file(credentials_1,localfilename)

<a id="part2"></a> 
# Part II - Analysis
<a id='prepare'></a>
## 1. Tone Analysis
 <a id='visualizations'></a>

#### Post Tone Dataframe

In [ ]:
#Create a new dataframe for analyzing the emotional tone from each Facebook entry
# 
post_tones = ["highestEmotionTone"]

#Create a new dataframe with tones
df_post_tones = df[post_tones]
#Aggregate the tone data for Analysis
tones = df_post_tones
tones = pd.DataFrame(tones.groupby('highestEmotionTone').size().reset_index(name='Posts'))
tones.head()

# PixieDust lets you visualize your data in just a few clicks using the display() API. You can find more info at https://ibm-cds-labs.github.io/pixiedust/displayapi.html. The following cell creates a DataFrame and uses the display() API to create a bar chart:

In [ ]:
#Use the Pixiedust library to easily visualize the data
display(tones)

# More Info.
For more information about PixieDust check out the following:
#### PixieDust Documentation: https://ibm-cds-labs.github.io/pixiedust/index.html
#### PixieDust GitHub Repo: https://github.com/ibm-cds-labs/pixiedust